In [ ]:
import h5py
import os, sys
import numpy as np
import trimesh
import pandas as pd
import matplotlib.pyplot as plt
from utils.Blendshape import DeformationComponents

path_to_3dgs = os.path.join(os.getcwd(), os.pardir, os.pardir,"gaussian-splatting")
path_to_output = os.path.join(path_to_3dgs, "output")
assert os.path.exists(path_to_output)

# session of id
session = "f336a291-bnotALLcam"
path_to_session = os.path.join(path_to_output, session)
assert os.path.exists(path_to_session)

# Deformation Learning via 3D Gaussian Splatting
- Apply following methhods on the trained 3D Gaussians based on multiface dataset
1. PCA
2. MiniBatch Sparse PCA

## Load 3D gaussian property

In [ ]:
# name_hdf5 = "f336a291-bnotALLcam_datamat_5509.hdf5"
name_hdf5 = "f336a291-bnotALLcam_datamat_87652.hdf5"
f = h5py.File(os.path.join(path_to_session, name_hdf5), 'r')
list(f.keys())

## target attributes
- xyz (center of Gaussians)
- normal (normal of Gaussians)
- rgb (color of Gaussians)
- opacity (opacity of Gaussians)
- scale (scale of Gaussians)
- rotation (rotation of Gaussians)

In [ ]:
target_attrib = "rgb"

In [ ]:
dataMat = f[target_attrib]
print(dataMat.shape)
average= np.mean(dataMat, axis = 0)
print(average.shape)
cent_X = dataMat - average
centX_std = np.std(cent_X)
print(cent_X.shape)

In [ ]:
dataMat.shape

## Load facemask

In [ ]:
from utils.Blendshape import FaceMask
from utils.pickel_io import dump_pckl, load_from_memory

# set the name of pickel file
mesh_loader = "trimesh"
path_to_samples = os.path.join(path_to_3dgs, "samples")
facemask_fname = "FaceMask_sample_subd2_face_22012024_12_51_trimesh.pkl"
# facemask_fname = "FaceMask_sample_face_22012024_13_25_trimesh.pkl"

facemask = load_from_memory(path_to_memory=path_to_samples, pickle_fname=facemask_fname)
print(facemask.bit_mask.flatten()[None, :].shape)

# create masked centralized datamatrix X

In [ ]:
masked_cent_X = cent_X * facemask.bit_mask.flatten()[None, :]
mesh = trimesh.load(os.path.join(path_to_samples, "sample_face.ply"), force = 'mesh')
tris = np.asarray(mesh.faces)

# Set the number of components

In [ ]:
Ncompos = 50
preScaleFactor = 1/centX_std
N_cent_X = cent_X * preScaleFactor
masked_N_cent_X = masked_cent_X * preScaleFactor

In [ ]:
N_cent_X.shape

#  1. PCA

In [ ]:
from sklearn.decomposition import PCA
D = masked_cent_X.shape[0]
pca = PCA(D)
pca.fit(N_cent_X.reshape(N_cent_X.shape[0], -1))

In [ ]:
Gamma = pca.components_.reshape(D, -1, 3) #right-hand side matrix of eigenvalue decomposition
Variance = pca.explained_variance_
Stds = np.sqrt(Variance)
Sigma = np.diag(pca.explained_variance_) #diagonal matrix which has eigenvectors in each diagonal entry
MEAN =  average # neutral face mesh (vertex coodinates of neutral face)

In [ ]:
masked_PCA_dc = Gamma * facemask.bit_mask[None, :]

In [ ]:
PCA_gamma = Gamma
PCA_variance = Variance

In [ ]:
Gamma.shape

In [ ]:
pca_RMSE = np.sqrt(((masked_N_cent_X.reshape(masked_N_cent_X.shape[0], -1) - np.dot(Sigma, (masked_PCA_dc).reshape(masked_PCA_dc.shape[0], -1)))**2).mean())

In [ ]:
print(pca_RMSE)

In [ ]:
pca_sparsity = np.sum(np.sqrt((masked_PCA_dc[:Ncompos, :]**2).sum(axis = 2)))

In [ ]:
print(pca_sparsity)

In [ ]:
pca_sparsity_level = np.mean(masked_PCA_dc==0)
print(pca_sparsity_level)

# visualize the flipped cumulative sum over obtained principal components
- As the corresponding value (y-axis) becomes lager, the component has larger global effect

$$\text{global influence}_{std_i} = \frac{std_i}{std_{max}}$$

In [ ]:
import pandas as pd

sum_std = Stds.max()
print(sum_std)
df_std = Stds/sum_std
cumsum_std=df_std
negative_cumsum_std = cumsum_std[:330]
plt.plot(negative_cumsum_std)
plt.ylabel("global influence of components")
plt.xlabel("No. of principal components")
# plt.xticks(np.arange(0, len(Stds)-1, 5))
plt.show()

# 2. MiniBatch SparcePCA

In [ ]:
from sklearn.decomposition import MiniBatchSparsePCA
mb_sparsePCA = MiniBatchSparsePCA(n_components=Ncompos, verbose=True)
est_MBSPCA = mb_sparsePCA.fit(N_cent_X.reshape(N_cent_X.shape[0], -1))

In [ ]:
C_mbspca = est_MBSPCA.components_.reshape(Ncompos, -1, 3) #right hand side V
W_mbspca = est_MBSPCA.transform(N_cent_X.reshape(N_cent_X.shape[0], -1)) #left hand side U

In [ ]:
masked_MBSPCA_dc = C_mbspca * facemask.bit_mask[None, :]

In [ ]:
MBSPCA_C = C_mbspca
MBSPCA_W = W_mbspca

In [ ]:
MBSPCA_C.shape

In [ ]:
mbspca_RMSE = np.sqrt(((masked_N_cent_X.reshape(masked_N_cent_X.shape[0], -1) - np.dot(W_mbspca, (masked_MBSPCA_dc).reshape(masked_MBSPCA_dc.shape[0], -1)))**2).mean())


In [ ]:
print(mbspca_RMSE)

In [ ]:
mbspca_sparsity = np.sum(np.sqrt((masked_MBSPCA_dc**2).sum(axis = 2)))

In [ ]:
print(mbspca_sparsity)

In [ ]:
mbspca_sparsity_level = np.mean(masked_MBSPCA_dc==0)
print(mbspca_sparsity_level)

# save deformation components

In [ ]:
pca_hdf5 = session+"_"+target_attrib+"_"+"PCA_dcs.hdf5"
PCA_deformationCompos = DeformationComponents(dataMat=dataMat, faceMask=facemask.bit_mask[None, :], MEAN = average, STD = centX_std, pcMat=PCA_gamma, coeffMat=Sigma, tris = tris, NofExp=dataMat.shape[0], NofFrame=5, NofVerts=int(dataMat.shape[1]/3))
PCA_deformationCompos.save_hdf5(path_to_save=path_to_session, fname = pca_hdf5)
pca_f = h5py.File(os.path.join(path_to_session, pca_hdf5), 'r')
list(pca_f.keys())
mbspca_hdf5 = session+"_"+target_attrib+"_"+"MBSPCA_dcs.hdf5"
MBSPCA_deformationCompos = DeformationComponents(dataMat=dataMat, faceMask=facemask.bit_mask[None, :], MEAN = average, STD = centX_std, pcMat=MBSPCA_C, coeffMat=MBSPCA_W, tris = tris, NofExp=dataMat.shape[0], NofFrame=5, NofVerts=int(dataMat.shape[1]/3))
MBSPCA_deformationCompos.save_hdf5(path_to_save=path_to_session, fname = mbspca_hdf5)
mbspca_f = h5py.File(os.path.join(path_to_session, mbspca_hdf5))
list(mbspca_f.keys())
